## Cargamos los datos de febrero 2020 / prueba

In [70]:
# Imports y configuración
import pandas as pd
import numpy as np
import os
import sys

# Permite importar desde ../src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from features.build_features import build_features
from modeling.predict import load_model, evaluate_model

In [71]:
taxi_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-02.parquet')
target_col = "tip_amount"
taxi_test = build_features(taxi_feb, target_col=target_col)
features = taxi_test.drop(columns=[target_col])
y_test = taxi_test[target_col]

y_test = (y_test > 0).astype(int)

C:\Users\raval\Dropbox\Magister Data Science\5TRIMESTRE\Desarrollo_Proyectos_y_Productos_Datos\Tarea_01\nyc_tips_classifier\src\features\build_features.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features + [target_col]] = df[features + [target_col]].astype("float32").fillna(-1.0)


In [72]:
taxi_test.head()

,tpep_dropoff_datetime,pickup_weekday,pickup_hour,work_hours,pickup_minute,passenger_count,trip_distance,trip_time,trip_speed,PULocationID,DOLocationID,RatecodeID,tip_amount
0,2020-02-01 00:30:32,5.0,0.0,0.0,17.0,1.0,2.60,777.0,0.003346,145.0,7.0,1.0,2.45
1,2020-02-01 01:05:36,5.0,0.0,0.0,32.0,1.0,4.80,1969.0,0.002438,45.0,61.0,1.0,6.30
2,2020-02-01 00:43:28,5.0,0.0,0.0,31.0,1.0,3.20,704.0,0.004545,186.0,140.0,1.0,1.00
3,2020-02-01 00:31:39,5.0,0.0,0.0,7.0,1.0,4.38,1444.0,0.003033,144.0,140.0,1.0,3.00
4,2020-02-01 01:01:29,5.0,0.0,0.0,51.0,1.0,2.28,586.0,0.003891,238.0,152.0,1.0,0.00


### Cargamos el modelo que guardamos y realizamos predicciones de prueba

In [74]:
model_path = os.path.join(os.getcwd(), "..", "models", "random_forest_model.joblib")
model = load_model(model_path)

In [75]:
model.feature_names_in_

array(['pickup_weekday', 'pickup_hour', 'work_hours', 'pickup_minute',
       'passenger_count', 'trip_distance', 'trip_time', 'trip_speed',
       'PULocationID', 'DOLocationID', 'RatecodeID'], dtype=object)

In [76]:
features = features[model.feature_names_in_]

In [77]:
from time import time

start = time()
f1_feb = evaluate_model(model, features, y_test)
print(f"F1-score para el conjunto de prueba (febrero 2020): {f1_feb:.4f}")
print(f"Tiempo total de evaluación: {time() - start:.2f} segundos")

F1-score para el conjunto de prueba (febrero 2020): 0.8401
Tiempo total de evaluación: 29.59 segundos


## Cargamos los datos de mayo 2020 / prueba

In [79]:
taxi_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-05.parquet')
target_col = "tip_amount"
taxi_test = build_features(taxi_feb, target_col=target_col)
features = taxi_test.drop(columns=[target_col])
y_test = taxi_test[target_col]

y_test = (y_test > 0).astype(int)

C:\Users\raval\Dropbox\Magister Data Science\5TRIMESTRE\Desarrollo_Proyectos_y_Productos_Datos\Tarea_01\nyc_tips_classifier\src\features\build_features.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features + [target_col]] = df[features + [target_col]].astype("float32").fillna(-1.0)


In [80]:
model.feature_names_in_

array(['pickup_weekday', 'pickup_hour', 'work_hours', 'pickup_minute',
       'passenger_count', 'trip_distance', 'trip_time', 'trip_speed',
       'PULocationID', 'DOLocationID', 'RatecodeID'], dtype=object)

In [81]:
features = features[model.feature_names_in_]

In [82]:
start = time()
f1_may = evaluate_model(model, features, y_test)
print(f"F1-score para el conjunto de prueba (febrero 2020): {f1_may:.4f}")
print(f"Tiempo total de evaluación: {time() - start:.2f} segundos")

F1-score para el conjunto de prueba (febrero 2020): 0.6803
Tiempo total de evaluación: 1.69 segundos


## Cargamos los datos de agosto 2020 / prueba

In [84]:
taxi_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-08.parquet')
target_col = "tip_amount"
taxi_test = build_features(taxi_feb, target_col=target_col)
features = taxi_test.drop(columns=[target_col])
y_test = taxi_test[target_col]

y_test = (y_test > 0).astype(int)

C:\Users\raval\Dropbox\Magister Data Science\5TRIMESTRE\Desarrollo_Proyectos_y_Productos_Datos\Tarea_01\nyc_tips_classifier\src\features\build_features.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features + [target_col]] = df[features + [target_col]].astype("float32").fillna(-1.0)


In [85]:
model.feature_names_in_

array(['pickup_weekday', 'pickup_hour', 'work_hours', 'pickup_minute',
       'passenger_count', 'trip_distance', 'trip_time', 'trip_speed',
       'PULocationID', 'DOLocationID', 'RatecodeID'], dtype=object)

In [86]:
features = features[model.feature_names_in_]

In [87]:
start = time()
f1_aug = evaluate_model(model, features, y_test)
print(f"F1-score para el conjunto de prueba (febrero 2020): {f1_aug:.4f}")
print(f"Tiempo total de evaluación: {time() - start:.2f} segundos")

F1-score para el conjunto de prueba (febrero 2020): 0.7495
Tiempo total de evaluación: 4.80 segundos


## Comparamos las métricas F1-score de los 3 períodos.

In [89]:
# Diccionario con los F1-scores
f1_scores = {
    "Febrero 2020": f1_feb,
    "Mayo 2020": f1_may,
    "Agosto 2020": f1_aug
}

# Convertimos a DataFrame
f1_df = pd.DataFrame.from_dict(f1_scores, orient='index', columns=["F1-score"])
f1_df.index.name = "Periodo"
f1_df = f1_df.round(4)

display(f1_df)


,F1-score
Periodo,
Febrero 2020,0.8401
Mayo 2020,0.6803
Agosto 2020,0.7495
